<a href="https://colab.research.google.com/github/Daz-Riza-Seriog/Tensorflow_ML/blob/main/2-Customise%20your%20Models/4-%20Week%204/7-Optimizing%20Performacne/Optimizing_Performance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
print(tf.__version__)

2.12.0


# Model subclassing and custom training loops

 ## Coding tutorials
 #### [1. Model subclassing](#coding_tutorial_1)
 #### [2. Custom layers](#coding_tutorial_2)
 #### [3. Automatic differentiation](#coding_tutorial_3)
 #### [4. Custom training loops](#coding_tutorial_4)
 #### [5. tf.function decorator](#coding_tutorial_5)

***
<a id="coding_tutorial_5"></a>
## tf.function decorator

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Softmax
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import reuters
import numpy as np
import matplotlib.pyplot as plt
import time

In [ ]:
# Define the custom layers and model

class MyLayer(Layer):

  def __init__(self,units):
    super(MyLayer, self).__init__()
    self.units = units

  def build(self,input_shape):
    self.w = self.add_weight(shape=(input_shape[-1],self.units),
                             initializer="random_normal",
                             name="kernel")
    self.b = self.add_weight(shape=(self.units,),
                             initializer="zeros",
                             name="bias")

  def call(self,inputs):
    return tf.matmul(inputs, self.w)+self.b

class MyDropout(Layer):

    def __init__(self, rate):
        super(MyDropout, self).__init__()
        self.rate = rate

    def call(self, inputs):
        # Define forward pass for dropout layer
        return tf.nn.dropout(inputs, rate=self.rate)

class MyModel(Model):

    def __init__(self, units_1, units_2, units_3):
        super(MyModel, self).__init__()
        # Define layers
        self.layer_1 = MyLayer(units=units_1)
        self.dropout_1 = MyDropout(0.5)
        self.layer_2 = MyLayer(units=units_2)
        self.dropout_2 = MyDropout(0.5)
        self.layer_3 = MyLayer(units=units_3)
        self.softmax = Softmax()

    def call(self, inputs):
        # Define forward pass
        x = self.layer_1(inputs)
        x = tf.nn.relu(x)
        x = self.dropout_1(x)
        x = self.layer_2(x)
        x = tf.nn.relu(x)
        x = self.dropout_2(x)
        x = self.layer_3(x)
        return self.softmax(x)

#### Build the model

In [ ]:
# Initialize a new model

model = MyModel(64,64,46)
print(model(tf.ones((1,10000))))
model.summary()

tf.Tensor(
[[0.03755128 0.02910216 0.0335117  0.00831797 0.0054531  0.05297132
  0.02788006 0.05265242 0.02862172 0.02237123 0.00556775 0.0328725
  0.01770614 0.0092869  0.0097434  0.00379581 0.03021331 0.00945698
  0.06763862 0.00499065 0.02085532 0.05432036 0.00545542 0.00917794
  0.00699623 0.0151393  0.00415886 0.01162654 0.02353209 0.00361663
  0.00839756 0.0446     0.02063499 0.0228554  0.02444356 0.01320631
  0.01445875 0.00748526 0.04991983 0.01773363 0.04038455 0.02422817
  0.01475259 0.03736133 0.01052155 0.00443286]], shape=(1, 46), dtype=float32)
Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 my_layer (MyLayer)          multiple                  640064    
                                                                 
 my_dropout (MyDropout)      multiple                  0         
                                                                 
 my_layer_1 (MyLayer)

In [ ]:
# Load the dataset

from tensorflow.keras.datasets import reuters

(train_data, train_labels), (test_data, test_labels) = reuters.load_data(num_words=10000, skip_top=50)

2110848/2110848 [==============================] - 0s 0us/step


In [ ]:
# Define the class names

class_names = ['cocoa','grain','veg-oil','earn','acq','wheat','copper','housing','money-supply',
   'coffee','sugar','trade','reserves','ship','cotton','carcass','crude','nat-gas',
   'cpi','money-fx','interest','gnp','meal-feed','alum','oilseed','gold','tin',
   'strategic-metal','livestock','retail','ipi','iron-steel','rubber','heat','jobs',
   'lei','bop','zinc','orange','pet-chem','dlr','gas','silver','wpi','hog','lead']

In [ ]:
# Print the class of the first sample

print("Label: {}".format(class_names[train_labels[0]]))

Label: earn


#### Get the dataset word index

In [ ]:
# Load the Reuters word index

word_to_index = reuters.get_word_index()

invert_word_index = dict([(value, key) for (key, value) in word_to_index.items()])
text_news = ' '.join([invert_word_index.get(i - 3, '?') for i in train_data[0]])

550378/550378 [==============================] - 0s 0us/step


In [ ]:
# Print the first data example sentence

print(text_news)

? ? ? ? ? ? result ? ? december acquisition ? space co ? expects earnings per share ? 1987 ? ? 15 ? ? 30 ? per share up ? 70 ? ? ? ? ? ? pretax ? should rise ? nine ? 10 ? ? ? six ? ? ? ? ? rental operation revenues ? 19 ? 22 ? ? ? 12 ? ? ? ? ? cash flow per share this ? should ? ? 50 ? three ? ? ?


#### Preprocess the data

In [ ]:
# Define a function that encodes the data into a 'bag of words' representation

def bag_of_words(text_samples, elements=10000):
    output = np.zeros((len(text_samples), elements))
    for i, word in enumerate(text_samples):
        output[i, word] = 1.
    return output

x_train = bag_of_words(train_data)
x_test = bag_of_words(test_data)

print("Shape of x_train:", x_train.shape)
print("Shape of x_test:", x_test.shape)

Shape of x_train: (8982, 10000)
Shape of x_test: (2246, 10000)


#### Define the loss function and optimizer



In [ ]:
# Define the categorical cross entropy loss and Adam optimizer

loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

def loss(model, x, y, wd):
    kernel_variables = []
    for l in model.layers:
        for w in l.weights:
            if 'kernel' in w.name:
                kernel_variables.append(w)
    wd_penalty = wd * tf.reduce_sum([tf.reduce_sum(tf.square(k)) for k in kernel_variables])
    y_ = model(x)
    return loss_object(y_true=y, y_pred=y_) + wd_penalty

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

In [ ]:
# Use the @tf.function decorator

@tf.function
def grad(model, inputs, targets, wd):
    with tf.GradientTape() as tape:
        loss_value = loss(model, inputs, targets, wd)
    return loss_value, tape.gradient(loss_value, model.trainable_variables)

#### Train the model

In [ ]:
# Re-run the training loop

# Implement the training loop

from tensorflow.keras.utils import to_categorical

start_time = time.time()

train_dataset = tf.data.Dataset.from_tensor_slices((x_train, train_labels))
train_dataset = train_dataset.batch(32)

# Keep results for plotting
train_loss_results = []
train_accuracy_results = []

num_epochs = 10
weight_decay = 0.005

for epoch in range(num_epochs):

  epoch_loss_avg = tf.keras.metrics.Mean()
  epoch_accuracy = tf.keras.metrics.CategoricalAccuracy()

  # Training Loop
  for x,y in train_dataset:
    #Optimize the model
    loss_value, grads = grad(model, x, y, weight_decay)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    # Compute current losses
    epoch_loss_avg(loss_value)
    # Compare predicted label to actual label
    epoch_accuracy(to_categorical(y), model(x))

  # End Epoch
  train_loss_results.append(epoch_loss_avg.result())
  train_accuracy_results.append(epoch_accuracy.result())

  print("Epoch {:03d}: Loss {:.3f}, Accuracy: {:.3%}".format(epoch,
                                                             epoch_loss_avg.result(),
                                                             epoch_accuracy.result()))

print("Duration :{:.3f}".format(time.time() - start_time))

Epoch 000: Loss 3.400, Accuracy: 45.669%
Epoch 001: Loss 1.937, Accuracy: 61.668%
Epoch 002: Loss 1.843, Accuracy: 65.832%
Epoch 003: Loss 1.802, Accuracy: 67.992%
Epoch 004: Loss 1.774, Accuracy: 68.849%
Epoch 005: Loss 1.757, Accuracy: 69.261%
Epoch 006: Loss 1.754, Accuracy: 69.962%
Epoch 007: Loss 1.735, Accuracy: 70.040%
Epoch 008: Loss 1.731, Accuracy: 70.586%
Epoch 009: Loss 1.732, Accuracy: 71.098%
Duration :177.759


#### Print the autograph code

In [ ]:
# Use tf.autograph.to_code to see the generated code

print(tf.autograph.to_code(grad.python_function))

def tf__grad(model, inputs, targets, wd):
    with ag__.FunctionScope('grad', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:
        do_return = False
        retval_ = ag__.UndefinedReturnValue()
        with ag__.ld(tf).GradientTape() as tape:
            loss_value = ag__.converted_call(ag__.ld(loss), (ag__.ld(model), ag__.ld(inputs), ag__.ld(targets), ag__.ld(wd)), None, fscope)
        try:
            do_return = True
            retval_ = (ag__.ld(loss_value), ag__.converted_call(ag__.ld(tape).gradient, (ag__.ld(loss_value), ag__.ld(model).trainable_variables), None, fscope))
        except:
            do_return = False
            raise
        return fscope.ret(retval_, do_return)

